In [4]:
import numpy as np
import pandas as pd
from scipy.stats import zscore

In [11]:
UOP_Celldensities =     pd.read_csv('../DataUOP/UOPfinal_celldensities.csv', index_col=0)
UOP_Function =          pd.read_csv('../DataUOP/UOPfinal_functional.csv', index_col=0)
UOP_Metavariables =     pd.read_csv('../DataUOP/UOPfinal_metavariables.csv', index_col=0)
UOP_Neighborhood =      pd.read_csv('../DataUOP/UOPfinal_neighborhood.csv', index_col=0)

UOP_data = {
    'UOP_Celldensities': UOP_Celldensities,
    'UOP_Function': UOP_Function,
    'UOP_Metavariables': UOP_Metavariables,
    'UOP_Neighborhood': UOP_Neighborhood
}

UOP_y = pd.read_csv('../DataUOP/UOPfinal_outcome.csv',index_col=0)
UOP_y = UOP_y.grade-1

In [12]:
from scipy.stats import ttest_ind, mannwhitneyu

for key, df in UOP_data.items():
    univariate = pd.DataFrame()
    for column in df.select_dtypes(include=['float64', 'int64']):
        x = df[column].dropna()
        outcome = UOP_y
        
        group1 = x[outcome == 0]  # Values in x corresponding to y.grade == 1
        group2 = x[outcome == 1]  # Values in x corresponding to y.grade == 2
        
        meandifference = group2.mean() - group1.mean()
        statistic, p_value_ttest = ttest_ind(group1, group2)
        statistic, p_value_mannwhitneyu = mannwhitneyu(group1, group2)
        univariate.loc[column, 'Statistic'] = statistic
        univariate.loc[column, 'meandifference higher-lower'] = meandifference
        univariate.loc[column, 'p-value ttest'] = p_value_ttest
        univariate.loc[column, 'p-value mannwhitneyu'] = p_value_mannwhitneyu

    filename = f'univariate_results_{key}.csv'
    univariate.to_csv(filename)

In [13]:
STA_Celldensities =     pd.read_csv('../DataStanford2/another_STA_celldensities.csv', index_col=0)
STA_Function =          pd.read_csv('../DataStanford2/another_STA_functional.csv', index_col=0)
STA_Metavariables =     pd.read_csv('../DataStanford2/another_STA_metavariables.csv', index_col=0)
STA_Neighborhood =      pd.read_csv('../DataStanford2/another_STA_neighborhood.csv', index_col=0)

STA_data = {
    'STA_Celldensities': STA_Celldensities,
    'STA_Function': STA_Function,
    'STA_Metavariables': STA_Metavariables,
    'STA_Neighborhood': STA_Neighborhood
}

STA_y = pd.read_csv('../DataStanford2/another_STA_outcome.csv',index_col=0)
STA_y = STA_y.grade-1

In [14]:
from scipy.stats import ttest_ind, mannwhitneyu

for key, df in STA_data.items():
    univariate = pd.DataFrame()
    for column in df.select_dtypes(include=['float64', 'int64']):
        x = df[column].dropna()
        outcome = STA_y
        
        group1 = x[outcome == 0]  # Values in x corresponding to y.grade == 1
        group2 = x[outcome == 1]  # Values in x corresponding to y.grade == 2
        
        meandifference = group2.mean() - group1.mean()
        statistic, p_value_ttest = ttest_ind(group1, group2)
        statistic, p_value_mannwhitneyu = mannwhitneyu(group1, group2)
        univariate.loc[column, 'Statistic'] = statistic
        univariate.loc[column, 'meandifference higher-lower'] = meandifference
        univariate.loc[column, 'p-value ttest'] = p_value_ttest
        univariate.loc[column, 'p-value mannwhitneyu'] = p_value_mannwhitneyu

    filename = f'univariate_results_{key}.csv'
    univariate.to_csv(filename)